In [10]:
%pip install transformers datasets evaluate ipywidgets scikit-learn numpy pandas tf-keras torch
%pip install tensorflow[and-cuda] 


  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
import tensorflow as tf
import pandas as pd
from transformers import TFBertForSequenceClassification, TFTrainingArguments, Trainer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt


In [27]:
# Load Dataset
df = pd.read_json('https://raw.githubusercontent.com/Wikidepia/SQuAD-id/refs/heads/master/data/train-SQuAD-id.json')

In [28]:
# Split Dataset into Train, Test, and Validation Sets
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Display the size of each set
print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

Train set size: 314
Validation set size: 39
Test set size: 40


In [29]:
def process_squad_data(df):
    squad_data = []
    for index, row in df.iterrows():
        entry = row['data']
        title = entry['title']
        for paragraph in entry['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                id_ = qa['id']
                if qa['is_impossible']:
                    answers = {'answer_start': [], 'text': []}
                else:
                    answers = {
                        'answer_start': [answer['answer_start'] for answer in qa['answers']],
                        'text': [answer['text'] for answer in qa['answers']]
                    }
                squad_data.append({
                    'answers': answers,
                    'context': context,
                    'id': id_,
                    'question': question,
                    'title': title
                })
    return squad_data

# Process the SQuAD data for train, validation, and test sets
train_squad_data = process_squad_data(train_df)
val_squad_data = process_squad_data(val_df)
test_squad_data = process_squad_data(test_df)

# Display the first entry of each set to verify
print(train_squad_data[0])
print(val_squad_data[0])
print(test_squad_data[0])

{'answers': {'answer_start': [187], 'text': ['Kekaisaran Turki, Ottoman Turkey atau Turki,']}, 'context': 'Kekaisaran Ottoman (/ ˈɒtəmən /; Turki Ottoman: دَوْلَتِ عَلِيّهٔ عُثمَانِیّه Devlet-i Aliyye-i Osmâniyye, Turki Modern: Osmanlı İmparatorluğu atau Osmanlı Devleti), juga dikenal sebagai Kekaisaran Turki, Ottoman Turkey atau Turki, Ottoman Turki 1299 oleh Oghuz Turks di bawah Osman I di Anatolia barat laut. Setelah penaklukan di Balkan oleh Murad I antara 1362 dan 1389, kesultanan Utsmaniyah diubah menjadi kekaisaran lintas benua dan penuntut kekhalifahan. Ottoman mengakhiri Kekaisaran Bizantium dengan penaklukan 1453 atas Konstantinopel oleh Mehmed sang Penakluk.', 'id': '572a11661d04691400779721', 'question': 'Kekaisaran Ottoman juga dikenal sebagai tiga nama lain apa?', 'title': 'Ottoman_Empire'}
{'answers': {'answer_start': [64], 'text': ['Republik Guinea-Bissau']}, 'context': 'Guinea-Bissau (i / ˈɡɪni bɪˈsaʊ /, GI-nee-bi-SOW), secara resmi Republik Guinea-Bissau (Portugis: Re

In [30]:
# Preprocess SQuAD Data
def preprocess_squad_data(squad_data):
    contexts = []
    questions = []
    answers = []
    
    for entry in squad_data:
        context = entry['context']
        question = entry['question']
        
        # Take the first answer for training purposes
        answer = entry['answers']['text'][0] if entry['answers']['text'] else None
        
        if answer:
            contexts.append(context)
            questions.append(question)
            answers.append(answer)
            
    return contexts, questions, answers

# Preprocess the SQuAD data for train, validation, and test sets
train_contexts, train_questions, train_answers = preprocess_squad_data(train_squad_data)
val_contexts, val_questions, val_answers = preprocess_squad_data(val_squad_data)
test_contexts, test_questions, test_answers = preprocess_squad_data(test_squad_data)

# Display the first entry of each set to verify
print(train_contexts[0], train_questions[0], train_answers[0])
print(val_contexts[0], val_questions[0], val_answers[0])
print(test_contexts[0], test_questions[0], test_answers[0])

Kekaisaran Ottoman (/ ˈɒtəmən /; Turki Ottoman: دَوْلَتِ عَلِيّهٔ عُثمَانِیّه Devlet-i Aliyye-i Osmâniyye, Turki Modern: Osmanlı İmparatorluğu atau Osmanlı Devleti), juga dikenal sebagai Kekaisaran Turki, Ottoman Turkey atau Turki, Ottoman Turki 1299 oleh Oghuz Turks di bawah Osman I di Anatolia barat laut. Setelah penaklukan di Balkan oleh Murad I antara 1362 dan 1389, kesultanan Utsmaniyah diubah menjadi kekaisaran lintas benua dan penuntut kekhalifahan. Ottoman mengakhiri Kekaisaran Bizantium dengan penaklukan 1453 atas Konstantinopel oleh Mehmed sang Penakluk. Kekaisaran Ottoman juga dikenal sebagai tiga nama lain apa? Kekaisaran Turki, Ottoman Turkey atau Turki,
Guinea-Bissau (i / ˈɡɪni bɪˈsaʊ /, GI-nee-bi-SOW), secara resmi Republik Guinea-Bissau (Portugis: República da Guiné-Bissau, diucapkan: [ʁeˈpublikɐ dɐ ɡiˈnɛ biˈsaw]), adalah sebuah negara di Barat Afrika. Ini mencakup 36.125 kilometer persegi (13.948 mil persegi) dengan perkiraan populasi 1.704.000. Apa nama resmi Guinea-B

In [ ]:
# Tokenize Data
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

# Tokenize the preprocessed data for train, validation, and test sets
train_encodings = tokenizer(train_contexts, train_questions, padding=True, truncation=True, return_tensors="pt")
val_encodings = tokenizer(val_contexts, val_questions, padding=True, truncation=True, return_tensors="pt")
test_encodings = tokenizer(test_contexts, test_questions, padding=True, truncation=True, return_tensors="pt")

# Display the first tokenized entry of each set to verify
print(train_encodings[0])
print(val_encodings[0])
print(test_encodings[0])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
